In [1]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.WARNING)
logging.root.level = logging.WARNING

import os
import jieba
import codecs
import time
start = time.clock()
documents = []
walk = os.walk('train')
for root ,dirs, files in walk:
    for name in files:
        #documents.append([])
        filename = os.path.join(root, name)
        try:
            f = open(filename)
            #f = codecs.open(filename,'r','utf-8','ignore')
            text = f.read()
        except:
            f = codecs.open(filename,'r','gbk','ignore')    
            text = f.read()
        finally:
            f.close()
        
        documents.append(text)
#documents = news_dataset.data

print ("In the dataset there are", len(documents), "textual documents")
end = time.clock()
print('Running time: %s Seconds'%(end-start))
#print ("And this is the first one:\n", documents[0])

In the dataset there are 14328 textual documents
Running time: 45.940007151193555 Seconds


In [2]:
import gensim
import time
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
start = time.clock()        
def load_stopwords():
    f_stop = open('stopwords.txt')
    try:
        f_stop_text = f_stop.read()

    finally:
        f_stop.close()

    f_stop_seg_list = f_stop_text.split('\n')
    return f_stop_seg_list

def tokenize(text):
    stopwords = load_stopwords()    

    return [token for token in jieba.cut(text) if token.strip() not in stopwords and len(token.strip()) > 1]

#print ("After the tokenizer, the previous document becomes:\n", tokenize(documents[0]))

processed_docs = [tokenize(doc) for doc in documents]
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print ("In the corpus there are", len(word_count_dict), "unique tokens")

end = time.clock()
print('Running time: %s Seconds'%(end-start))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 1.872 seconds.
DEBUG:jieba:Loading model cost 1.872 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In the corpus there are 254680 unique tokens
Running time: 561.5696837013134 Seconds


In [3]:
word_count_dict.filter_extremes(no_below=20, no_above=0.1) # word must appear >10 times, and no more than 20% documents
print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")

bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

bow_doc1 = bag_of_words_corpus[0]

#print ("Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1)
print
for i in range(5):
    print ("In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1]))
print ("...")

After filtering, in the corpus there are only 20174 unique tokens
In the document, topic_id 161 (word "强迫") appears 1 time[s]
In the document, topic_id 170 (word "竞争对手") appears 1 time[s]
In the document, topic_id 190 (word "掌握") appears 2 time[s]
In the document, topic_id 269 (word "付款") appears 1 time[s]
In the document, topic_id 386 (word "合作") appears 1 time[s]
...


In [4]:
#LDA

start = time.clock()
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100, iterations=1000,id2word=word_count_dict, passes=5)
end = time.clock()
print('Running time: %s Seconds'%(end-start))

Running time: 336.64838952411696 Seconds



init(self, corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01)


In [5]:
print ("Log perplexity of the model is", lda_model.log_perplexity(bag_of_words_corpus))

Log perplexity of the model is -8.65940147904


In [12]:
for index, score in sorted(lda_model[bag_of_words_corpus[34]], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index, lda_model.print_topic(index, 5)))

Score: 0.4273499875659849	 Index: 35	 
Topic: 0.031*我国 + 0.010*高原 + 0.010*药品 + 0.007*生产 + 0.007*工程
Score: 0.16775903302428025	 Index: 57	 
Topic: 0.104*英国 + 0.090*德国 + 0.052*欧洲 + 0.022*美元 + 0.020*15000
Score: 0.13868251583648938	 Index: 56	 
Topic: 0.060*网络 + 0.040*网站 + 0.032*计算机 + 0.027*平台 + 0.022*软件
Score: 0.08624531873290552	 Index: 68	 
Topic: 0.165*人才 + 0.108*招聘 + 0.047*职位 + 0.044*求职 + 0.033*行业
Score: 0.05227497278432184	 Index: 30	 
Topic: 0.053*合同 + 0.046*协议 + 0.035*签署 + 0.024*双方 + 0.022*律师
Score: 0.04935269211029111	 Index: 66	 
Topic: 0.057*增长 + 0.033*收入 + 0.031*亿美元 + 0.024*亿元 + 0.020*万人
Score: 0.038393163857376574	 Index: 40	 
Topic: 0.103*导弹 + 0.016*采购 + 0.014*研发 + 0.012*in + 0.009*生产
Score: 0.02245418947833482	 Index: 99	 
Topic: 0.124*队员 + 0.061*昨天 + 0.046*战胜 + 0.033*这次 + 0.028*调整
Score: 0.010208126610013444	 Index: 3	 
Topic: 0.019*精神 + 0.018*生命 + 0.013*话题 + 0.013*意义 + 0.011*死刑


In [14]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.WARNING)
logging.root.level = logging.WARNING
import os
import jieba
import codecs
import time
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

start = time.clock()
def read_documents(file):
    documents = []
    walk = os.walk(file)
    for root ,dirs, files in walk:
        for name in files:
            filename = os.path.join(root, name)
            try:
                f = open(filename)
                #f = codecs.open(filename,'r','utf-8','ignore')
                text = f.read()
            except:
                f = codecs.open(filename,'r','gbk','ignore')    
                text = f.read()
            finally:
                f.close()

            documents.append(text)
    return documents
#documents = news_dataset.data
documents = read_documents('train')
print ("In the dataset there are", len(documents), "textual documents")
#print ("And this is the first one:\n", documents[0])
        
def load_stopwords():
    f_stop = open('stopwords.txt')
    try:
        f_stop_text = f_stop.read()

    finally:
        f_stop.close()

    f_stop_seg_list = f_stop_text.split('\n')
    return f_stop_seg_list

def tokenize(text):
    stopwords = load_stopwords()    
    return [token for token in jieba.cut(text) if token.strip() not in stopwords and len(token.strip()) > 1]

#print ("After the tokenizer, the previous document becomes:\n", tokenize(documents[0]))


processed_docs = [tokenize(doc) for doc in documents]
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print ("In the corpus there are", len(word_count_dict), "unique tokens")

word_count_dict.filter_extremes(no_below=20, no_above=0.1)
print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")

bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

#bow_doc1 = bag_of_words_corpus[0]
#print ("Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1)
#print
#for i in range(5):
#    print ("In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1]))
#print ("...")


# LDA mono-core
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100,iterations=1000, id2word=word_count_dict, passes=5)

# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

#_ = lda_model.print_topics(-1)


end = time.clock()
print('Running time: %s Seconds'%(end-start))
    

In the dataset there are 14328 textual documents
In the corpus there are 254680 unique tokens
After filtering, in the corpus there are only 20174 unique tokens
Running time: 955.7922709637319 Seconds


In [16]:
print ("Log perplexity of the model is", lda_model.log_perplexity(bag_of_words_corpus))

Log perplexity of the model is -8.66070309045


In [17]:
start = time.clock()
#for index, score in sorted(lda_model[bag_of_words_corpus[0]], key=lambda tup: -1*tup[1]):
#    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index, lda_model.print_topic(index, 5)))


unseen_documents = read_documents('test')
print ("In the test dataset there are", len(unseen_documents), "unseen textual documents")

processed_docs = [tokenize(doc) for doc in unseen_documents]
#word_count_dict = gensim.corpora.Dictionary(processed_docs)
#print ("In the corpus there are", len(word_count_dict), "unique tokens")

#word_count_dict.filter_extremes(no_below=20, no_above=0.1)
#print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")


bow_vector = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]


end = time.clock()
print('Running time: %s Seconds'%(end-start))

In the test dataset there are 3582 unseen textual documents
Running time: 161.79975180185875 Seconds


In [120]:
print ("待测文本内容:\n", unseen_documents[1])
print('\n')
for index, score in sorted(lda_model[bow_vector[1]], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index,lda_model.print_topic(index, 5)))

待测文本内容:
 &nbsp&nbsp&nbsp&nbsp贵州茅台酒股份有限公司(以下简称"本公司")于近日收到贵州省国有资产监督管理委员会(黔国资产权函〔2006〕94号)《关于对贵州茅台酒股份有限公司股权分置改革有关问题的批复》,本公司股权分置改革方案已获贵州省国有资产监督管理委员会的批准。
&nbsp&nbsp&nbsp&nbsp特此公告！
&nbsp&nbsp&nbsp&nbsp贵州茅台酒股份有限公司董事会
&nbsp&nbsp&nbsp&nbsp2006年5月9日


Score: 0.6866545526529277	 Index: 91	 
Topic: 0.049*现役 + 0.042*股东 + 0.038*股份 + 0.029*股权 + 0.018*有限公司
Score: 0.13830074505642279	 Index: 14	 
Topic: 0.046*民族 + 0.032*皇帝 + 0.023*高原 + 0.018*贵州 + 0.017*起义
Score: 0.05809926462240962	 Index: 54	 
Topic: 0.027*一体化 + 0.020*探索 + 0.019*生产 + 0.011*军兵种 + 0.011*每年
Score: 0.03805623563852788	 Index: 73	 
Topic: 0.056*万元 + 0.036*有限公司 + 0.033*深圳 + 0.030*贷款 + 0.024*广东
Score: 0.031105381620195865	 Index: 52	 
Topic: 0.029*贝尔 + 0.024*导师 + 0.022*武汉 + 0.022*博士生 + 0.020*冒险
Score: 0.021672709298405885	 Index: 9	 
Topic: 0.140*韩国 + 0.050*该机 + 0.020*得分 + 0.020*联盟 + 0.017*约翰逊


In [31]:
print(bag_of_words_corpus[0])

[(161, 1), (170, 1), (190, 2), (269, 1), (386, 1), (522, 1), (583, 1), (672, 3), (1151, 1), (1217, 2), (1250, 1), (1251, 1), (1356, 2), (1439, 1), (1454, 2), (1528, 1), (1751, 1), (1783, 2), (1788, 1), (1830, 1), (1946, 3), (1972, 3), (1975, 1), (1980, 1), (2091, 1), (2116, 2), (2133, 4), (2139, 1), (2181, 1), (2200, 1), (2374, 1), (2408, 1), (2423, 1), (2535, 1), (2587, 2), (2602, 6), (2657, 1), (2971, 2), (2978, 1), (2998, 1), (3021, 2), (3045, 2), (3060, 1), (3076, 1), (3181, 1), (3432, 2), (3465, 1), (3483, 1), (3597, 1), (3709, 1), (3722, 1), (3741, 1), (3772, 1), (3801, 8), (3890, 1), (4117, 1), (4178, 1), (4254, 1), (4409, 1), (4483, 1), (4548, 3), (4570, 1), (4620, 1), (4791, 1), (4793, 2), (4803, 1), (4804, 1), (4814, 1), (4870, 1), (4878, 2), (4896, 4), (4985, 1), (4987, 5), (5037, 1), (5055, 2), (5176, 2), (5287, 1), (5393, 1), (5472, 17), (5551, 1), (5571, 1), (5699, 1), (5719, 1), (5808, 1), (5810, 1), (5848, 1), (5989, 1), (6026, 1), (6217, 1), (6220, 1), (6412, 1), (6613

In [27]:
doc_topic = lda_model.get_document_topics(bag_of_words_corpus[0])
doc_topic

[(0, 0.023131753827911005),
 (2, 0.041930141072334393),
 (7, 0.11325677145996337),
 (16, 0.056483535724439225),
 (31, 0.011318338601286813),
 (34, 0.445717998128764),
 (37, 0.013819062013953247),
 (46, 0.027223576090984948),
 (60, 0.01378317274386166),
 (66, 0.015881645353275728),
 (71, 0.013560863841236974),
 (72, 0.020357836397192998),
 (73, 0.031611287442607147),
 (90, 0.1247690897026431)]

In [24]:
# LDA mono-core
lda_model1 = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100,iterations=1000, id2word=word_count_dict, passes=5,minimum_probability=0)


In [37]:
doc_topic2 = lda_model1.get_document_topics(bow_vector[0])
doc_topic2

[(0, 2.0491803278688521e-05),
 (1, 2.0491803278688521e-05),
 (2, 2.0491803278688521e-05),
 (3, 2.0491803278688521e-05),
 (4, 2.0491803278688521e-05),
 (5, 2.0491803278688521e-05),
 (6, 2.0491803278688521e-05),
 (7, 2.0491803278688521e-05),
 (8, 2.0491803278688521e-05),
 (9, 2.0491803278688521e-05),
 (10, 2.0491803278688521e-05),
 (11, 2.0491803278688521e-05),
 (12, 2.0491803278688521e-05),
 (13, 2.0491803278688521e-05),
 (14, 2.0491803278688521e-05),
 (15, 2.0491803278688521e-05),
 (16, 2.0491803278688521e-05),
 (17, 2.0491803278688521e-05),
 (18, 2.0491803278688521e-05),
 (19, 0.024784114029842454),
 (20, 2.0491803278688521e-05),
 (21, 2.0491803278688521e-05),
 (22, 0.0024243944244010615),
 (23, 2.0491803278688521e-05),
 (24, 2.0491803278688521e-05),
 (25, 2.0491803278688521e-05),
 (26, 2.0491803278688521e-05),
 (27, 2.0491803278688521e-05),
 (28, 2.0491803278688521e-05),
 (29, 2.0491803278688521e-05),
 (30, 2.0491803278688521e-05),
 (31, 2.0491803278688521e-05),
 (32, 2.0491803278688

In [49]:
doc_topic1 = lda_model1.get_document_topics(bag_of_words_corpus[0])
doc_topic1

[(0, 0.003627125389142569),
 (1, 2.3809523809523841e-05),
 (2, 2.3809523809523841e-05),
 (3, 0.0071120184837482024),
 (4, 2.3809523809523841e-05),
 (5, 2.3809523809523841e-05),
 (6, 2.3809523809523841e-05),
 (7, 0.0028248420667943766),
 (8, 2.3809523809523841e-05),
 (9, 2.3809523809523841e-05),
 (10, 2.3809523809523841e-05),
 (11, 2.3809523809523841e-05),
 (12, 2.3809523809523841e-05),
 (13, 2.3809523809523841e-05),
 (14, 0.32578412746609992),
 (15, 2.3809523809523841e-05),
 (16, 2.3809523809523841e-05),
 (17, 2.3809523809523841e-05),
 (18, 0.0059156027738695906),
 (19, 0.0073844637522411912),
 (20, 2.3809523809523841e-05),
 (21, 2.3809523809523841e-05),
 (22, 2.3809523809523841e-05),
 (23, 2.3809523809523841e-05),
 (24, 2.3809523809523841e-05),
 (25, 2.3809523809523841e-05),
 (26, 2.3809523809523841e-05),
 (27, 2.3809523809523841e-05),
 (28, 2.3809523809523841e-05),
 (29, 2.3809523809523841e-05),
 (30, 0.0024580114457410538),
 (31, 2.3809523809523841e-05),
 (32, 2.3809523809523841e-05

In [73]:
x_train = []
for i in range(len(documents)):
    x_train.append([])
    doc_topic = lda_model1.get_document_topics(bag_of_words_corpus[i])
    for topic in doc_topic:
        x_train[i].append(topic[1])
x_train

[[0.0036270475090560556,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  0.0071116786006176855,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  0.0028249914497624332,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  0.32578159641722843,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  0.0059154095579339629,
  0.0074180533399263813,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  0.0024580090078116161,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  0.1302605403933807,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  2.3809523809523851e-05,
  0.05561131898252511,
  2.3809523809523851e-05,
  

In [74]:
x_train[0]

[0.0036270475090560556,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.0071116786006176855,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.0028249914497624332,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.32578159641722843,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.0059154095579339629,
 0.0074180533399263813,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.0024580090078116161,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.1302605403933807,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.05561131898252511,
 2.3809523809523851e-05,
 2.3809523809523851e-05,
 0.0052415917070

In [75]:
x_test = []
for i in range(len(unseen_documents)):
    x_test.append([])
    doc_topic = lda_model1.get_document_topics(bow_vector[i])
    for topic in doc_topic:
        x_test[i].append(topic[1])
x_test[0]

[2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 0.024459159412786043,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 0.0024270539108399106,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 0.0033405376360075905,
 0.0058336785809948583,
 0.0021159159622479774,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.0491803278688514e-05,
 2.049

In [76]:
y_train =[]
for i in range(9):
    y_train.extend([i]*1592)
y_test = []
for i in range(9):
    y_test.extend([i]*398)
print(len(y_train),len(y_test))

14328 3582


In [80]:
from sklearn import svm
#---------------------------------------------#
#svm模型
svc1 = svm.SVC(kernel='rbf')
svc1.fit(x_train,y_train) 
svc1_pred = svc1.predict(x_test)
svc1_score = svc1.score(x_test,y_test)
#---------------------------------------------#
print(svc1_score)

0.646286990508


In [83]:
from sklearn import svm
#---------------------------------------------#
#svm模型
svc2 = svm.SVC(kernel='poly')
svc2.fit(x_train,y_train) 
svc2_pred = svc2.predict(x_test)
svc2_score = svc2.score(x_test,y_test)
#---------------------------------------------#


0.646286990508


In [121]:
from sklearn import svm
#---------------------------------------------#
#svm模型
svc3 = svm.SVC(kernel='sigmoid')
svc3.fit(x_train,y_train) 
svc3_pred = svc3.predict(x_test)
svc3_score = svc3.score(x_test,y_test)
#---------------------------------------------#

In [84]:
print(svc2_score)

0.140982691234


In [122]:
print(svc3_score)

0.111111111111


In [81]:
from sklearn import feature_extraction
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier 
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn import tree
from sklearn.naive_bayes import MultinomialNB
#---------------------------------------------#
knnclf = KNeighborsClassifier()
knnclf.fit(x_train,y_train) 
knn_pred = knnclf.predict(x_test)
knn_score = knnclf.score(x_test,y_test)
#knn模型
#---------------------------------------------#



#---------------------------------------------#
#svm模型
svc = svm.SVC(kernel='linear')
svc.fit(x_train,y_train) 
svc_pred = svc.predict(x_test)
svc_score = svc.score(x_test,y_test)
#---------------------------------------------#



#---------------------------------------------#
#tree模型
tre = tree.DecisionTreeClassifier()
tre.fit(x_train,y_train) 
tre_pred = tre.predict(x_test)
tre_score = tre.score(x_test,y_test)
#---------------------------------------------#



#---------------------------------------------#
#bayes模型
bayes = MultinomialNB(alpha = 0.01)
bayes.fit(x_train,y_train) 
bayes_pred = bayes.predict(x_test)
bayes_score = bayes.score(x_test,y_test)
#---------------------------------------------#


#print(np.mean(doc_class_predicted == y_test))

#准确率与召回率
#precision, recall, thresholds = precision_recall_curve(y_test, clf.predict(x_test))
#answer = clf.predict_proba(x_test)[:,1]
#report = answer > 0.5
#print(classification_report(y_test, report, target_names = ['neg', 'pos']))

In [82]:
print("knn score:",knn_score)
print("svm score:",svc_score)
print("tree score:",tre_score)
print("bayes score:",bayes_score)


knn score: 0.801507537688
svm score: 0.809603573423
tree score: 0.721105527638
bayes score: 0.796761585706


In [104]:
import numpy as np
print(np.mean(svc_pred == y_test))
print(svc_pred)

0.809603573423
[0 0 1 ..., 2 8 8]











































C000007 汽车 C000008 财经 C000010 IT C000013 健康 C000014 体育 C000016 旅游 C000020 教育 C000022 招聘 C000023 文化 C000024 军事


In [97]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = svc_pred
target_names = ['财经', '网络', '健康','体育','旅游','教育','招聘','文化','军事']
print(classification_report(y_true, y_pred, target_names=target_names))
                 

             precision    recall  f1-score   support

         财经       0.87      0.81      0.83       398
         网络       0.80      0.78      0.79       398
         健康       0.83      0.80      0.81       398
         体育       0.96      0.99      0.97       398
         旅游       0.88      0.79      0.83       398
         教育       0.90      0.76      0.82       398
         招聘       0.75      0.82      0.78       398
         文化       0.57      0.67      0.62       398
         军事       0.80      0.87      0.84       398

avg / total       0.82      0.81      0.81      3582



In [106]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = knn_pred
target_names = ['财经', '网络', '健康','体育','旅游','教育','招聘','文化','军事']
print(classification_report(y_true, y_pred, target_names=target_names))

             precision    recall  f1-score   support

         财经       0.94      0.75      0.83       398
         网络       0.84      0.73      0.78       398
         健康       0.75      0.83      0.79       398
         体育       0.97      0.99      0.98       398
         旅游       0.83      0.82      0.82       398
         教育       0.81      0.75      0.78       398
         招聘       0.74      0.75      0.75       398
         文化       0.58      0.75      0.66       398
         军事       0.85      0.84      0.84       398

avg / total       0.81      0.80      0.80      3582



In [103]:
from sklearn.metrics import classification_report
y_true = y_test
y_pred = bayes_pred
target_names = ['财经', '网络', '健康','体育','旅游','教育','招聘','文化','军事']
print(classification_report(y_true, y_pred, target_names=target_names))

             precision    recall  f1-score   support

         财经       0.91      0.78      0.84       398
         网络       0.78      0.77      0.78       398
         健康       0.83      0.76      0.80       398
         体育       0.97      0.99      0.98       398
         旅游       0.83      0.77      0.80       398
         教育       0.88      0.74      0.80       398
         招聘       0.75      0.79      0.77       398
         文化       0.55      0.69      0.61       398
         军事       0.76      0.88      0.82       398

avg / total       0.81      0.80      0.80      3582



In [85]:
help(svm.SVC)

Help on class SVC in module sklearn.svm.classes:

class SVC(sklearn.svm.base.BaseSVC)
 |  C-Support Vector Classification.
 |  
 |  The implementation is based on libsvm. The fit time complexity
 |  is more than quadratic with the number of samples which makes it hard
 |  to scale to dataset with more than a couple of 10000 samples.
 |  
 |  The multiclass support is handled according to a one-vs-one scheme.
 |  
 |  For details on the precise mathematical formulation of the provided
 |  kernel functions and how `gamma`, `coef0` and `degree` affect each
 |  other, see the corresponding section in the narrative documentation:
 |  :ref:`svm_kernels`.
 |  
 |  .. The narrative documentation is available at http://scikit-learn.org/
 |  
 |  Parameters
 |  ----------
 |  C : float, optional (default=1.0)
 |      Penalty parameter C of the error term.
 |  
 |  kernel : string, optional (default='rbf')
 |       Specifies the kernel type to be used in the algorithm.
 |       It must be one of 

In [101]:
dir(sklearn.metrics)

['SCORERS',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'accuracy_score',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'auc',
 'average_precision_score',
 'base',
 'brier_score_loss',
 'classification',
 'classification_report',
 'cluster',
 'completeness_score',
 'confusion_matrix',
 'consensus_score',
 'coverage_error',
 'euclidean_distances',
 'explained_variance_score',
 'f1_score',
 'fbeta_score',
 'get_scorer',
 'hamming_loss',
 'hinge_loss',
 'homogeneity_completeness_v_measure',
 'homogeneity_score',
 'jaccard_similarity_score',
 'label_ranking_average_precision_score',
 'log_loss',
 'make_scorer',
 'matthews_corrcoef',
 'mean_absolute_error',
 'mean_squared_error',
 'median_absolute_error',
 'mutual_info_score',
 'normalized_mutual_info_score',
 'pairwise',
 'pairwise_distances',
 'pairwise_distances_argmin',
 'pairwise_distances_argmin_min',
 'pairwise_fast',
 'pairwise_